In [1]:
import torch
import torch.optim as optim
import torch_geometric
from torch.nn.functional import relu, sigmoid, softmax, mse_loss
from torch.nn import Linear, Module, Dropout, MSELoss, CrossEntropyLoss, BatchNorm1d

from tqdm import tqdm

from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, GATv2Conv

import pandas as pd
import numpy as np

import os
import pickle
import gzip
import optuna

/home/kuangr/inoue019/.conda/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
device = 0
device = torch.device("cuda:{}".format(device) if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cpu')

In [4]:
train = pd.read_csv('../BindingDB/train.csv', index_col=0)
val = pd.read_csv('../BindingDB/val.csv', index_col=0)
test = pd.read_csv('../BindingDB/test.csv', index_col=0)


# In[7]:


print('Train dim:', train.shape)
print('val dim:', val.shape)
print('test dim:', test.shape)

Train dim: (15979, 3)
val dim: (2300, 3)
test dim: (4472, 3)


In [5]:
def get_protein_dataloader(proteins, batch_size=100, is_cmap=True):
    if is_cmap:
        dataset = [ ]
        for i in proteins:
            print(i)
            dataset.append(torch.load('../cmap/{}.pt'.format(i)))
    else:
        dataset = [torch.load('../protein_graphs/{}.pt'.format(i)) for i in proteins]
        
    return DataLoader(dataset, batch_size=batch_size)

In [6]:
batch_size = 10
is_cmap = True

protein_train_loader = get_protein_dataloader(train['Target_ID'], batch_size, is_cmap)

P00915
P00918
P00918
P29274
P08588
P13945
P00533
P00533
P05132
O75116
Q9UM73
Q9UM73
Q9UM73
Q9UM73
Q9UM73
Q9UM73
O14965
O14965
O14965
Q96GD4
Q96GD4
Q96GD4
Q96GD4
Q96GD4
Q96GD4
Q9UQB9
Q9UQB9
Q9UQB9
Q9UQB9
Q9UQB9
Q9UQB9
Q9UQB9
Q9UQB9
Q9UQB9
P14920
P30530
P30530
P30530
P30530
P30530
P30530
P30530
P30530
Q13873
Q13873
Q13873
Q9UQ88
Q9UQ88
Q9UQ88
P50613
P50613
P50613
P50613
P50613
P50613
P50750
P50750
Q9HAZ1
Q9HAZ1
Q9HAZ1
Q9HAZ1
Q9HAZ1
Q13164
Q9H2J4
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
P24941
Q9Z429
Q9Z429
P9WPL0
P9WPL4
P9WPL0
P9WPP0
P9WPL4
P9WPP0
P9WPL4
P9WPN4
P9WPN5
P0A881
P19921
P58154
P58154
O74036
O74036
Q9BY41
Q9BY41
O60885
P36888
P10721
P09619
Q92772
P00519
P15056
P07333
P21802
Q13153
P07949
P35968
O95372
Q13164
Q13547
Q92769
Q13547
Q92769
Q13547
Q92769
P35354
P35354
O75530
Q9HYR3
Q9HYR3
P08238
Q05816
P01375
P15121
P49841
P15056
P15056
P13631
P19754
P10826
P10826
P33535
P30969
P41144
P97266
P32300
P10276
P28704

Q8TDC3
P00519
Q9UEE5
P00519
P42336
Q86Y07
Q9Y616
Q9Y6R4
P24723
Q15349
P10721
Q9BZL6
Q04912
P07949
Q9Y6R4
O75582
P10721
P24723
Q15746
P00533
Q12852
Q86YV6
O75582
P22607
P00519
P10721
P28482
P00533
P00533
P42684
P00519
P22607
P00519
Q15349
O43353
P54764
Q2M2I8
Q09013
Q15349
P00519
O75582
Q09013
Q09013
P00533
Q8TDC3
Q14004
O15075
P00519
Q14012
O75582
P10721
Q9BYT3
Q8TDC3
Q96RR4
Q9UEE5
P08581
Q8N4C8
P07949
P07949
Q13976
P00519
P53355
P42336
P07949
Q2M2I8
P10721
Q9UQM7
Q9BZL6
P53355
Q59H18
P10721
P11802
P78362
P06213
Q6DT37
Q15418
Q6DT37
P00519
P22607
O14757
Q8N4C8
P00519
Q8N4C8
P24723
O60331
P36888
P07949
Q13164
P00519
Q8N4C8
P10721
Q9H2K8
P00533
P36888
P36888
P24723
P36888
P07949
P24723
Q59H18
Q9P2K8
P07949
P36888
Q13163
P42336
P51957
Q6DT37
P00533
P07949
P00519
P10721
P78362
P10721
P48730
Q59H18
P00533
P29597
P36888
P00519
P00519
P21709
Q8TD19
Q6DT37
P21709
P36888
P00519
P00533
P36888
P42336
P00533
P10721
P36888
Q86Y07
O15111
P00533
Q9Y6R4
P51956
Q2M2I8
P00519
Q15349
Q05655
P00519
P29597

Q6DT37
Q04912
P00519
Q8N4C8
P54760
Q9P2K8
P00533
P24723
P00519
Q16832
P10721
Q15746
Q9UF33
Q9UEE5
Q04912
P00533
Q59H18
Q04912
P00519
O95382
P00519
P36888
P10721
Q9UF33
P10721
P00533
P29322
Q99640
P78362
Q13163
P00519
P10721
O60331
Q15831
P24723
P00519
P10721
P10721
Q59H18
Q9UQ88
P08581
P00533
P00533
P9WI81
O75582
Q09013
Q8N752
O95835
P11802
Q13523
P23458
Q9H3Y6
P36888
P00533
Q15746
Q13554
Q8TDC3
Q15746
O14757
O14757
O75582
P53355
Q9Y616
Q6DT37
Q6DT37
P36888
P00533
P00533
P00519
P08581
P24723
P00533
Q15418
P22607
Q2M2I8
Q9BZL6
P00533
O75582
P29597
Q15349
O75582
P10721
P19784
P10721
P42681
P00533
P29597
Q13976
P10721
P00533
P00519
P21709
P15056
O14757
P00533
P53355
Q9BZL6
Q13976
P06213
Q9NY57
O14757
Q9UQ88
Q15418
Q9Y616
P36888
Q15418
Q2M2I8
P24723
Q15569
Q13554
P00533
Q09013
Q04912
Q59H18
Q9UQM7
Q9H1R3
P00519
Q15349
Q15746
P36888
P10721
Q9UF33
P00533
P78362
Q9UL54
P07949
Q15746
P00519
P10721
Q9UF33
Q15746
P00519
Q8WXR4
P29597
Q9UF33
P21709
O14936
P78362
Q8N4C8
Q09013
Q96GD4
Q8WXR4
Q96RR4

P24723
Q9UBE8
Q15418
P10721
Q9BZL6
P49841
O75582
P15056
P24723
P22455
P10721
P00519
P36888
P07949
P00519
O75914
P00533
P42336
P10721
Q9UK32
P17612
P15056
O14757
O75716
Q96GD4
P37023
P35916
P00533
P36888
P00533
Q9Y616
O43293
Q15349
P00519
P36888
Q96GD4
P10721
Q13976
P10721
Q96GD4
O14757
P00533
Q15418
P08922
P42336
P49840
Q9UK32
Q13131
P21802
P50750
P07949
O14757
P54753
Q09013
P15056
Q13976
Q15418
O96013
P22607
P00533
P42336
P00519
O43293
P00519
O75582
P00519
Q9Y616
Q15418
Q13976
O75582
P31749
P42336
P07949
P49840
Q9BZL6
Q9UK32
O14965
P10721
P29597
P00533
Q9BZL6
O95835
P36888
Q9UK32
P42336
P42336
O43293
Q9UK32
P42336
P22607
Q9UM73
P24723
Q15418
P00519
P49840
P11799
P36894
P21709
P07949
P42336
P15056
P00519
Q9BZL6
O43293
Q09013
P15056
Q15418
Q09013
P07949
Q9UK32
P00533
P29320
Q02763
Q9P289
Q9UK32
Q96GD4
P16234
Q15418
P00519
P42336
Q13976
P36888
Q13976
P45985
P10721
Q9Y616
P22694
P10721
Q9Y616
Q9UK32
O14757
P07949
P07949
P00533
O14757
P08631
P00533
O75582
O43293
Q13976
Q9Y616
Q9HC98
Q05655

Q8NE63
P51617
Q00536
P16234
P09619
P04629
Q16288
P17948
P20794
P42336
Q5VT25
P08238
P08238
P00918
Q8N1Q1
P16234
P09619
P07949
P07333
P04798
Q02750
Q02750
Q02750
P14920
P14920
P14920
P14920
P14920
P07900
Q8N5S9
P54764
P54753
O75747
O94768
P36896
Q9UM73
P41240
Q9Y6M4
P29317
Q16513
Q04759
Q13237
Q13188
Q13043
Q06418
Q86Y07
Q12852
Q9Y2U5
Q9NYY3
Q15208
Q96PY6
Q9UKI8
O94921
O60331
Q9UEW8
Q96BR1
Q14164
P14616
O00444
Q9UIK4
Q9UPE1
Q9HC98
Q9BQI3
Q9NRM7
Q9H2G2
Q14680
O60285
Q9Y2H1
Q9H0K1
Q9NRP7
Q9UL54
Q9H2K8
Q9P289
Q8NEV4
P29323
Q7KZI7
P42685
P08631
P35968
P53667
P07948
P42679
Q99759
Q16584
P51955
P51956
Q13153
P09619
O15530
O00750
P11309
P48736
Q9UBF8
P17612
Q15139
P27361
P31152
Q16659
Q13164
P45983
O15264
P52564
P19525
Q15835
P08922
P45985
Q96SB4
P51957
O76039
P43405
P42680
P37173
P33981
P42681
P29597
P30291
P78356
Q9Y6E0
Q92630
Q8IVH8
Q14012
Q8IW41
Q13546
Q13523
Q8IWQ3
Q15208
Q96PY6
O94921
O60331
Q9UHD2
Q96BR1
Q14164
P14616
O00444
Q9UIK4
Q9UPE1
Q9HC98
Q9UK32
Q9NRM7
P57058
Q8IYT8
Q14680
O60285

...

Q9UKE5
Q9H0K1
Q9NRP7
Q9NWZ3
Q9UL54
Q9H2K8
Q9P289
Q8NEV4
Q7KZI7
Q9NSY1
Q9NRH2
Q9NY57
Q9P0L2
P49840
Q8NI60
Q9P286
Q8IU85
Q96NX5
P29322
P07949
P57078
O60674
O00329
P53350
P51817
O95747
P31749
P29320
P16591
O14976
Q02156
Q13464
P12931
P35590
P07947
Q9Y243
Q08881
P53671
O15146
Q9H422
O94806
P20794
P42338
P54646
Q05655
Q16513
Q04759
Q13237
Q13188
Q13043
Q15569
Q06418
Q86Y07
Q12852
O00506
Q9Y2U5
O14578
Q15208
Q96PY6
Q9UKI8
O60331
Q9UEW8
Q9UHD2
Q96BR1
P54756
P32298
Q8TD19
P80192
Q9C098
P9WI81
Q9H3Y6
Q8TDR2
Q13627
Q8TDX7
P48730
Q96Q40
Q8WTQ7
Q13163
O43318
O95819
Q8NG66
Q86Z02
P06241
Q8IY84
Q05397
Q14289
P57059
Q8NFD2
P54756
Q32MK0
P32298
Q6P3R8
P29376
P43403
P0C1S8
P0C264
P61075
P49759
Q16288
Q12866
Q86UE8
Q16832
P37023
Q06187
P52333
P10721
Q15831
Q02763
P08069
P36888
P36896
P36894
P41240
Q9Y6M4
P29317
P54762
P54753
P54760
O15197
O75747
P51812
O95382
O95835
O43283
O15075
Q00535
O60674
O00329
P53350
P51817
O95747
Q99683
Q13882
O94804
Q96L34
Q9BRS2
Q8N5S9
Q02779
P51955
Q13177
O75914
P09619
O00750

Q8TD19
Q9H1R3
P9WI81
Q9H3Y6
Q8TDR2
Q13627
Q8TDX7
Q15139
P27361
P31152
Q16659
Q13164
P45983
Q15759
O15264
P46734
P52564
P19525
Q15835
P08922
Q8IU85
P29322
P07949
P57078
Q9Y243
Q08881
P53671
O15146
Q9H422
Q99683
Q13882
O94804
Q9Y5S2
Q16620
O94921
O60331
Q9UEW8
Q9UHD2
Q96BR1
Q14164
P14616
O00444
Q9UIK4
Q9HC98
Q9UK32
P57058
O60285
Q9UPZ9
Q9Y2H1
Q9H0K1
P42684
P31749
P00533
P29320
P16591
O14976
P06239
Q02156
P12931
P35590
P07947
Q9Y243
Q08881
P53671
O15146
Q9H422
O94806
Q99683
Q13882
O94804
Q16620
P16234
Q16816
P54646
Q16513
Q13188
Q9Y463
P42345
O00329
P53350
P51817
O95747
P31749
P07333
P29320
P16591
P09769
O14976
P06239
Q02156
Q13464
P12931
P07947
Q9Y243
Q08881
P53671
O15146
Q9H422
O94806
P20794
Q99683
Q13882
O94804
Q9Y5S2
Q16620
P16234
Q16816
Q05655
Q16513
Q04759
Q13237
Q13188
Q13043
Q15569
Q06418
Q86Y07
Q12852
Q9Y2U5
Q9NYY3
O14578
Q15208
Q96PY6
Q9UKI8
O94921
O60331
Q9UEW8
Q9UHD2
Q96BR1
P14616
O00444
Q9UIK4
Q9UPE1
Q00535
P42345
O60674
O00329
P53350
P51817
O95747
P42684
P31749
P07333
P16591

Q9H4B4
Q9UM73
P54764
Q07912
Q9H1R3
O43353
P31152
P42338
Q5VT25
P49674
Q9H2G2
P48736
Q16512
P42684
P68400
P36896
P08069
P58154
Q16548
Q70I53
Q13547
Q9UBN7
P02752
P02752
O60885
O60885
O60885
Q15059
Q15059
O60885
Q58F21
P25440
Q15059
O43318
O43318
O43318
O43318
P08235
P37231
Q07869
Q03181
P02768
P30291
P30291
P0C1S8
Q99640
Q99640
O60674
P53350
P52333
P30291
Q99640
Q99640
P30291
P00918
P42338
P48736
P48736
P48736
O00329
O00329
O00329
P42345
P42345
O00750
O00750
O00750
Q8NEB9
Q9UBF8
P42336
P42336
P42336
P42338
P48736
P42345
O00750
Q9UBF8
Q8NEB9
P42336
P42338
P50053
P02766
P02766
O60674
O60674
O60674
P23458
Q8N1Q1
P00918
P00918
P00918
Q16790
P00918
Q8N1Q1
P00918
P00918
Q16790
P22303
P22303
P16257
P16257
P21728
Q63495
Q15109
Q15109
Q15109
P02766
Q9H8M2
P68400
P00519
P11473
P11473
P15090
P15090
P47900
P21836
P18031
P25440
Q15059
P25440
P30542
P30542
P30542
P30542
P42260
P22756
P22756
P00918
P0A9J4
P26637
P35218
P00918
Q13546
Q16850
P10828
P00533
P14867
P14867
P36544
P70605
P70605
P15056
P15056

P34976
P30994
P30994
P30994
P30547
P70536
Q9JI35
P30994
Q9JI35
P34995
P30994
P43144
P14842
P30994
P30994
P14842
P14842
Q9Y5N1
Q9UBM7
P23944
P43140
Q3U3Q1
O43293
P49761
Q06187
P49760
Q9NRP7
O14976
P29320
Q96RR4
Q9UEE5
Q13043
P43405
P00519
P06241
Q9H2G2
P49759
Q2M2I8
Q07912
P51955
P07947
P07948
Q13153
Q9JI10
P41240
Q99640
P06239
Q9Y2H1
O60674
P30553
P08173
P08172
P34976
P25095
P25095
P50129
P25095
P25095
P30098
O70212
P34976
P31389
P31389
P0A2Y6
P47747
P30542
P70536
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P04278
P02768
P02768
P02768
O43613
P9WP01
P9WP01
P9WP01
P9WP01
Q7Z2W7
P58154
P58154
P08183
P08183
P08183
P08183
P08183
P08183
P09429
Q9GZT9
Q9GZT9
Q9GZT9
P00519
P10415
P20701
P10415
P08581
P08581
P07445
P07445
P07445
P61964
P61964
P61964
P61964
P61964
Q03181
Q07869
P37231
P37231
Q8N1Q1
Q8N1Q1
O43570
O43570
O43570
P43166
P43166
P00915

/home/kuangr/inoue019/.conda/envs/py39/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
